In [27]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import numpy as np


# Load the dataframes
train = pd.read_csv("setEntrenamiento.csv")
test = pd.read_csv("setTesteo.csv")
train = train.drop('Unnamed: 0', axis = 1)
train['created_on'] = pd.to_datetime(train['created_on'])
test = test.drop('Unnamed: 0', axis = 1)
test['created_on'] = pd.to_datetime(test['created_on'])

In [28]:
train = train.drop('barrio', axis =1)

In [29]:
def fechaNumerica(x):
    return 10000*x.year + 100*x.month + x.day

In [30]:
dicc_tipos_prop = {"apartment" : 1, "house" : 2, "PH" : 3, "store" : 4}
dicc_tipos_zona = {"Bs.As. G.B.A. Zona Norte": 1, "Bs.As. G.B.A. Zona Oeste": 2, "Bs.As. G.B.A. Zona Sur": 3, "Capital Federal": 4 }
train.loc[:, 'property_type'] = train.property_type.apply(lambda x: dicc_tipos_prop.get(x))
train.loc[:, 'zona'] = train.zona.apply(lambda x: dicc_tipos_zona.get(x))
train.loc[:, 'created_on'] = train.created_on.apply(fechaNumerica)

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f382d344710>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/xgboost/core.py", line 324, in __del__
    _check_call(_LIB.XGDMatrixFree(self.handle))
AttributeError: 'DMatrix' object has no attribute 'handle'


In [31]:
# Organize our data for training
X = train.drop(["price_aprox_usd"], axis=1)
Y = train["price_aprox_usd"]
x_test = test
X, X_Val, Y, Y_Val = train_test_split(X, Y)

In [32]:

# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=1) 

grid = GridSearchCV(xgb, params)
grid.fit(X, Y)

# Print the r2 score
print(r2_score(Y_Val, grid.best_estimator_.predict(X_Val))) 

# Save the file
y_test = grid.best_estimator_.predict(x_test)
results_df = pd.DataFrame(data={'y':y_test}) 
ids = test_df["ID"]
joined = pd.DataFrame(ids).join(results_df)
joined.to_csv("mercedes.csv", index=False)

# This scored 0.5563 for me on the LB
# Let me know if you have any remarks or have any ideas for improving it.

KeyboardInterrupt: 